name: frequency_model.ipynb

This notebook contains all the code for prototyping and running the simple feedforward neural network trained on NT frequency values. Additionally, we test that our high R^2 and Spearman values make sense by running a naive random predictor and testing the evaluation.

At the very end, there is code to test the automation of running the script counterpart to this notebook, `MLME-v2.0-frequency_model.py`.

---

In [ ]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/Shareddrives/NRC_Amii_Agronomics_Project/nrc-ml-plant-genomics/'

Mounted at /content/drive


In [ ]:
import argparse
import keras
import warnings, logging
import numpy as np
import pandas as pd
import datetime, os
import json
import random
import shutil

from keras.models import Sequential, load_model, model_from_json
from keras.layers import Input, Dense, Conv1D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping  # https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/

from sklearn.metrics import r2_score
from scipy.stats import spearmanr  # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html

warnings.filterwarnings('ignore')
logging.disable(1000)

In [ ]:
def get_model(in_dim):  # initializes model architecture
    mdl = Sequential()

    mdl.add(Dense(12, input_dim=in_dim, activation='relu'))
    # mdl.add(Dense(24, activation='relu'))
    # mdl.add(Dense(8, activation='sigmoid'))

    mdl.add(Dense(1, activation='linear'))

    return mdl

activations = ["relu"]
n = 3  # 1 for nt, 2 for dnt, 3 for tnt

learning_rate_ = 0.005
patience_ = 20
num_epochs_ = 500
batch_size_ = 32

In [ ]:
ls

data/         freq.ipynb  main.ipynb  new_models/  time.ipynb
experiments/  logs/       model/      src/


In [ ]:
df = pd.read_csv("data/processed/hidra_chloroplast_70.csv")

In [ ]:
df

,organelle,start_coords,end_coords,sequence,control_raw_coverage,treatment_raw_coverage,control_norm_coverage,treatment_norm_coverage,target,A,T,C,G,set
0,NC_016734.1,0,145,AATCATAATAACTTGGTCCCGGGCATCACGGGCGAACGACGGGAAT...,998,316,338.31,405.52,0.26,39,40,40,26,test
1,NC_016734.1,5,150,TAATAACTTGGTCCCGGGCATCACGGGCGAACGACGGGAATTGAAC...,998,318,338.31,408.08,0.27,40,40,39,26,test
2,NC_016734.1,10,155,ACTTGGTCCCGGGCATCACGGGCGAACGACGGGAATTGAACCCGCG...,998,318,338.31,408.08,0.27,38,38,42,27,test
3,NC_016734.1,15,160,GTCCCGGGCATCACGGGCGAACGACGGGAATTGAACCCGCGATGGT...,998,318,338.31,408.08,0.27,40,36,42,27,test
4,NC_016734.1,20,165,GGGCATCACGGGCGAACGACGGGAATTGAACCCGCGATGGTGAATT...,998,318,338.31,408.08,0.27,41,39,39,26,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24687,NC_016734.1,152695,152840,GAGAAAATCGAATTTCTTTCTTCGTCTTAAAAAAAATAGGAGTTAA...,2239,555,759.00,712.22,-0.09,66,39,18,22,val
24688,NC_016734.1,152700,152845,AATCGAATTTCTTTCTTCGTCTTAAAAAAAATAGGAGTTAATTAAC...,2115,514,716.97,659.61,-0.12,66,40,19,20,val
24689,NC_016734.1,152705,152850,AATTTCTTTCTTCGTCTTAAAAAAAATAGGAGTTAATTAACTGTGA...,2062,492,699.00,631.38,-0.15,64,42,18,21,val
24690,NC_016734.1,152710,152855,CTTTCTTCGTCTTAAAAAAAATAGGAGTTAATTAACTGTGACACGT...,2034,484,689.51,621.11,-0.15,62,39,21,23,val


In [ ]:
# df = df[["A", "T", "C", "G", "target", "set"]]
# df.loc[:,["A", "T", "C", "G"]] = df.loc[:,["A", "T", "C", "G"]]/145

In [ ]:
nts = ["A", "T", "C", "G"]  # list of single nucleotides
dnts = [nt1+nt2 for nt1 in nts for nt2 in nts]  # list of dinucleotides
tnts = [nt1+nt2+nt3 for nt1 in nts for nt2 in nts for nt3 in nts]  # list of dinucleotides

In [ ]:
dnt_freqs = []
for i in range(df.shape[0]):
  sqnc_freq = []
  for item in dnts:
    sqnc_freq.append(df.loc[i, "sequence"].count(item))
  dnt_freqs.append(sqnc_freq)
df[dnts] = dnt_freqs

tnt_freqs = []
for i in range(df.shape[0]):
  sqnc_freq = []
  for item in tnts:
    sqnc_freq.append(df.loc[i, "sequence"].count(item))
  tnt_freqs.append(sqnc_freq)
df[tnts] = tnt_freqs

In [ ]:
df.head()

,organelle,start_coords,end_coords,sequence,control_raw_coverage,treatment_raw_coverage,control_norm_coverage,treatment_norm_coverage,target,A,T,C,G,set,AA,AT,AC,AG,TA,TT,TC,TG,CA,CT,CC,CG,GA,GT,GC,GG,AAA,AAT,AAC,AAG,ATA,ATT,ATC,ATG,ACA,ACT,...,TCA,TCT,TCC,TCG,TGA,TGT,TGC,TGG,CAA,CAT,CAC,CAG,CTA,CTT,CTC,CTG,CCA,CCT,CCC,CCG,CGA,CGT,CGC,CGG,GAA,GAT,GAC,GAG,GTA,GTT,GTC,GTG,GCA,GCT,GCC,GCG,GGA,GGT,GGC,GGG
0,NC_016734.1,0,145,AATCATAATAACTTGGTCCCGGGCATCACGGGCGAACGACGGGAAT...,998,316,338.31,405.52,0.26,39,40,40,26,test,10,15,12,0,11,8,11,9,10,11,8,8,6,5,6,6,1,6,3,0,4,3,7,1,3,5,...,4,3,4,0,2,3,1,3,1,5,4,0,5,4,1,1,2,2,3,3,3,0,2,3,4,1,1,0,1,1,2,1,1,1,2,2,1,2,3,3
1,NC_016734.1,5,150,TAATAACTTGGTCCCGGGCATCACGGGCGAACGACGGGAATTGAAC...,998,318,338.31,408.08,0.27,40,40,39,26,test,11,14,12,0,12,8,10,9,9,11,8,8,6,5,6,6,2,6,3,0,4,3,6,1,3,5,...,3,3,4,0,2,3,1,3,1,4,4,0,5,4,1,1,2,2,3,3,3,0,2,3,4,1,1,0,1,1,2,1,1,1,2,2,1,2,3,3
2,NC_016734.1,10,155,ACTTGGTCCCGGGCATCACGGGCGAACGACGGGAATTGAACCCGCG...,998,318,338.31,408.08,0.27,38,38,42,27,test,9,13,14,0,10,8,10,9,10,11,8,9,6,5,7,6,2,5,3,0,3,3,6,1,4,5,...,3,3,4,0,2,3,1,3,1,4,5,0,5,4,1,1,2,2,3,3,3,0,3,3,4,1,1,0,1,1,2,1,1,1,2,2,1,2,3,3
3,NC_016734.1,15,160,GTCCCGGGCATCACGGGCGAACGACGGGAATTGAACCCGCGATGGT...,998,318,338.31,408.08,0.27,40,36,42,27,test,10,13,13,1,10,7,10,8,12,10,8,9,6,5,8,5,2,5,3,0,3,3,6,1,4,4,...,3,3,4,0,2,3,1,2,2,4,5,1,5,3,1,1,2,2,3,3,3,0,3,3,4,1,1,0,1,1,2,1,3,1,2,2,1,1,3,3
4,NC_016734.1,20,165,GGGCATCACGGGCGAACGACGGGAATTGAACCCGCGATGGTGAATT...,998,318,338.31,408.08,0.27,41,39,39,26,test,10,15,13,1,11,8,9,8,12,10,7,8,6,4,8,5,2,6,3,0,4,4,6,1,4,4,...,3,3,3,0,2,3,1,2,2,4,5,1,5,3,1,1,2,2,2,2,3,0,3,2,4,1,1,0,1,1,1,1,3,1,2,2,1,1,3,3


In [ ]:
if n == 1:
  # SINGLE NUCLEOTIDES
  train_df = df[df.set == "train"]
  X_train = np.array(train_df[nts])
  y_train = np.array(train_df["target"].tolist())

  val_df = df[df.set == "val"]
  X_val = np.array(val_df[nts])
  y_val = np.array(val_df["target"].tolist())

  test_df = df[df.set == "test"]
  X_test = np.array(test_df[nts])
  y_test = np.array(test_df["target"].tolist())
elif n == 2:
  # DINUCLEOTIDES
  train_df = df[df.set == "train"]
  X_train = np.array(train_df[dnts])
  y_train = np.array(train_df["target"].tolist())

  val_df = df[df.set == "val"]
  X_val = np.array(val_df[dnts])
  y_val = np.array(val_df["target"].tolist())

  test_df = df[df.set == "test"]
  X_test = np.array(test_df[dnts])
  y_test = np.array(test_df["target"].tolist())
elif n == 3:
  # TRINUCLEOTIDES
  train_df = df[df.set == "train"]
  X_train = np.array(train_df[tnts])
  y_train = np.array(train_df["target"].tolist())

  val_df = df[df.set == "val"]
  X_val = np.array(val_df[tnts])
  y_val = np.array(val_df["target"].tolist())

  test_df = df[df.set == "test"]
  X_test = np.array(test_df[tnts])
  y_test = np.array(test_df["target"].tolist())
else:
  assert False

---
Start from here

In [ ]:
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
dir_path = "experiments/simple_"+date+"_"+str(learning_rate_)+"_"+str(batch_size_)+"_"+str(num_epochs_)+"_"+str(patience_)

In [ ]:
model = get_model(n)

In [ ]:
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_63 (Dense)             (None, 12)                780       
_________________________________________________________________
dense_64 (Dense)             (None, 1)                 13        
Total params: 793
Trainable params: 793
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate_), loss='mean_squared_error')

In [ ]:
logdir = os.path.join(dir_path, "logs")
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=patience_)
mc_callback = ModelCheckpoint(dir_path+'/best_model.h5', monitor='val_loss', save_best_only=True)

history = model.fit(X_train, y_train,
                    epochs=num_epochs_,
                    batch_size=batch_size_,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/500
619/619 [==============================] - 2s 2ms/step - loss: 1.3799 - val_loss: 0.2042
Epoch 2/500
619/619 [==============================] - 1s 2ms/step - loss: 0.1730 - val_loss: 0.2285
Epoch 3/500
619/619 [==============================] - 1s 1ms/step - loss: 0.1658 - val_loss: 0.2143
Epoch 4/500
619/619 [==============================] - 1s 2ms/step - loss: 0.1530 - val_loss: 0.2087
Epoch 5/500
619/619 [==============================] - 1s 1ms/step - loss: 0.1451 - val_loss: 0.2126
Epoch 6/500
619/619 [==============================] - 1s 1ms/step - loss: 0.1358 - val_loss: 0.2052
Epoch 7/500
619/619 [==============================] - 1s 1ms/step - loss: 0.1336 - val_loss: 0.2164
Epoch 8/500
619/619 [==============================] - 1s 1ms/step - loss: 0.1278 - val_loss: 0.2169
Epoch 9/500
619/619 [==============================] - 1s 1ms/step - loss: 0.1279 - val_loss: 0.1959
Epoch 10/500
619/619 [==============================] - 1s 1ms/step - loss: 0.1204 - val_lo

In [ ]:
hist_df = pd.DataFrame(history.history) 
hist_df.to_csv(dir_path+'/training_history.csv')

saved_model = load_model(dir_path+'/best_model.h5')

In [ ]:
with open(dir_path+'/modelsummary.txt', 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
    f.write("\nLayer Activation Functions:\n")
    for i in range(len(activations)):
      f.write("Layer"+str(i)+": "+activations[i]+"\n")

In [ ]:
with open(dir_path+"/results.csv", "w") as f:
  f.write(",R2,spearman\n")
  f.write("train,"+str(r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))+","+str(spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0])[0])+"\n")
  f.write("val,"+str(r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))+","+str(spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0])[0])+"\n")
  f.write("test,"+str(r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))+","+str(spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0])[0]))

In [ ]:
# R2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))
print()
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.5188323956704354
Val 0.34709466993739857
Test 0.40327359910660576

Train SpearmanrResult(correlation=0.722130419813937, pvalue=0.0)
Val SpearmanrResult(correlation=0.6043848085175052, pvalue=3.872851139739585e-244)
Test SpearmanrResult(correlation=0.6229170981134238, pvalue=1.44792002147141e-263)


---
Test With Random

In [ ]:
# set range from test data
max_val = y_train.max()
min_val = y_train.min()

In [ ]:
y_pred = np.array([random.uniform(min_val, max_val) for _ in range(len(y_test))])

In [ ]:
y_test

array([ 0.26,  0.27,  0.27, ...,  0.05,  0.01, -0.03])

In [ ]:
y_pred

array([-0.50094196, -1.68000804,  0.88937023, ...,  0.85508843,
       -0.08243427, -0.77596516])

In [ ]:
print(r2_score(y_test, y_pred))
print(spearmanr(y_test, y_pred)[0])

-3.088578714250951
-0.001619156183863346


---
Test Auto Run

In [ ]:
import tensorflow
print(tensorflow.__version__)

2.5.0


In [ ]:
from subprocess import run
import sys
import os
import time

In [ ]:
run(["python", "src/MLME-frequency_model-v2.0.py"], capture_output=True)

CompletedProcess(args=['python', 'src/freq.py'], returncode=1, stdout=b'experiments/nucfreq_20210707-192135_nucTrueFalseFalse_lay12-0-0-1_lr0.002_bs512\nEpoch 1/500\n', stderr=b'2021-07-07 19:21:30.723221: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0\n2021-07-07 19:21:34.824158: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1\n2021-07-07 19:21:34.936172: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected\n2021-07-07 19:21:34.936255: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a71cb73dcd70): /proc/driver/nvidia/version does not exist\n2021-07-07 19:21:35.047214: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.\n2021-07-07 19:21:35.047280: I tensorflow/core/prof

In [ ]:
!python src/MLME-frequency_model-v2.0.py -mo 1 -di 1 -tr 0 -l1 24

2021-07-07 19:37:26.515864: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-07 19:37:33.422442: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-07 19:37:33.433613: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-07-07 19:37:33.433669: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a71cb73dcd70): /proc/driver/nvidia/version does not exist
experiments/nucfreq_20210707-193733_nuc110_lay24-0-0-1_lr0.002_bs512
2021-07-07 19:37:33.467864: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-07-07 19:37:33.467921: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-07-07 19:37:33.477287: I tensorflow/core/profiler

In [ ]:
!python src/MLME-frequency_model-v2.0.py

2021-07-13 19:50:26.652913: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-13 19:50:31.965075: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-13 19:50:32.034491: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-07-13 19:50:32.034551: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dd25d4084167): /proc/driver/nvidia/version does not exist
2021-07-13 19:50:32.124245: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-07-13 19:50:32.124293: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-07-13 19:50:32.134219: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-07-13 

In [ ]:
!python src/MLME-frequency_model-v2.0.py

2021-07-13 19:50:53.102566: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-13 19:50:56.104346: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-13 19:50:56.115623: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-07-13 19:50:56.115688: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dd25d4084167): /proc/driver/nvidia/version does not exist
2021-07-13 19:50:56.141253: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-07-13 19:50:56.141301: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-07-13 19:50:56.151079: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-07-13 